Cube representation
        0  1
        2  3
16 17   8  9    4  5    20 21
18 19   10 11   6  7    22 23
        12 13
        14 15

Color representation
        0
    4   2   1   5
        3

In [2]:
from os import defpath
from random import choice, randint
from sys import argv
from time import time
import copy
import time
import matplotlib.pyplot as plt

# Permutations for each move
MOVES_LIST = {
    "U":  [2,   0,   3,   1,  20,  21,   6,   7,   4,   5,  10,  11,  12,  13,  14,  15,   8,   9,  18,  19,  16,  17,  22,  23],
    "U'": [1,   3,   0,   2,   8,   9,   6,   7,  16,  17,  10,  11,  12,  13,  14,  15,  20,  21,  18,  19,   4,   5,  22,  23],
    "R":  [0,   9,   2,  11,   6,   4,   7,   5,   8,  13,  10,  15,  12,  22,  14,  20,  16,  17,  18,  19,   3,  21,   1,  23],
    "R'": [0,  22,   2,  20,   5,   7,   4,   6,   8,   1,  10,   3,  12,   9,  14,  11,  16,  17,  18,  19,  15,  21,  13,  23],
    "F":  [0,   1,  19,  17,   2,   5,   3,   7,  10,   8,  11,   9,   6,   4,  14,  15,  16,  12,  18,  13,  20,  21,  22,  23],
    "F'": [0,   1,   4,   6,  13,   5,  12,   7,   9,  11,   8,  10,  17,  19,  14,  15,  16,   3,  18,   2,  20,  21,  22,  23],
    "D":  [0,   1,   2,   3,   4,   5,  10,  11,   8,   9,  18,  19,  14,  12,  15,  13,  16,  17,  22,  23,  20,  21,   6,   7],
    "D'": [0,   1,   2,   3,   4,   5,  22,  23,   8,   9,   6,   7,  13,  15,  12,  14,  16,  17,  10,  11,  20,  21,  18,  19],
    "L":  [23,  1,  21,   3,   4,   5,   6,   7,   0,   9,   2,  11,   8,  13,  10,  15,  18,  16,  19,  17,  20,  14,  22,  12],
    "L'": [8,   1,  10,   3,   4,   5,   6,   7,  12,   9,  14,  11,  23,  13,  21,  15,  17,  19,  16,  18,  20,   2,  22,   0],
    "B":  [5,   7,   2,   3,   4 , 15,   6,  14,   8,   9,  10,  11,  12,  13,  16,  18,   1,  17,   0,  19,  22,  20,  23,  21],
    "B'": [18, 16,   2,   3,   4,   0,   6,   1,   8,   9,  10,  11,  12,  13,   7,   5,  14,  17,  15,  19,  21,  23,  20,  22],

    "U2": [3, 2, 1, 0, 16, 17, 6, 7, 20, 21, 10, 11, 12, 13, 14, 15, 4, 5, 18, 19, 8, 9, 22, 23],
    "R2": [0, 13, 2, 15, 7, 6, 5, 4, 8, 22, 10, 20, 12, 1, 14, 3, 16, 17, 18, 19, 11, 21, 9, 23],
    "F2": [0, 1, 13, 12, 19, 5, 17, 7, 11, 10, 9, 8, 3, 2, 14, 15, 16, 6, 18, 4, 20, 21, 22, 23],
    "D2": [0, 1, 2, 3, 4, 5, 18, 19, 8, 9, 22, 23, 15, 14, 13, 12, 16, 17, 6, 7, 20, 21, 10, 11],
    "L2": [12, 1, 14, 3, 4, 5, 6, 7, 23, 9, 21, 11, 0, 13, 2, 15, 19, 18, 17, 16, 20, 10, 22, 8],
    "B2": [15, 14, 2, 3, 4, 18, 6, 16, 8, 9, 10, 11, 12, 13, 1, 0, 7, 17, 5, 19, 23, 22, 21, 20]
}

# Potential Values for distance in Heuristics Function and PDDL
# Sticker Indices for each corner pieces
CORNER_LIST = [(10, 12, 19), ( 6, 11, 13),
               ( 2,  8, 17), ( 3,  4,  9),
               (14, 18, 23), ( 7, 15, 22),
               ( 0, 16, 21), ( 1,  5, 20)]

# 3D Coordinates for each corner
COORDINATES_LIST = [(0, 0, 0), (0, 1, 0), (1, 0, 0), (1, 1, 0),
                    (0, 0, 1), (0, 1, 1), (1, 0, 1), (1, 1, 1)]

class Cube2x2:

    # Function: Initialize Cube
    def __init__(self, cube = "WWWW RRRR GGGG YYYY OOOO BBBB", movesList=[], depth=0):
        self.movesList = movesList
        self.depth = depth
        self.state = self.checkCube(cube)

    # Function: Check Cube String
    def checkCube(self, state):
        state = state.replace(" ", "")
        state = state.upper()
        if len(state) != 24:
            raise ValueError("State must have only 24 Stickers.")
        colors = ["W", "R", "G", "Y", "O", "B"]
        for color in colors:
            if state.count(color) != 4:
                raise ValueError("State must exactly have 4 Stickers of each Color.")
        return state

    # Function: Print Cube
    def print(self):
        print("    {}{}              \n".format(self.state[0], self.state[1]))
        print("    {}{}              \n".format(self.state[2], self.state[3]))
        print(" {}{} {}{} {}{} {}{}  \n".format(self.state[16], self.state[17], self.state[8], self.state[9], self.state[4], self.state[5], self.state[20], self.state[21]))
        print(" {}{} {}{} {}{} {}{}  \n".format(self.state[18], self.state[19], self.state[10], self.state[11], self.state[6], self.state[7], self.state[22], self.state[23]))
        print("    {}{}              \n".format(self.state[12], self.state[13]))
        print("    {}{}              \n".format(self.state[14], self.state[15]))

    # Function: Return Current State
    def returnCurrentState(self):
        return self.state

    # Function: Check if Goal State Reached
    def checkIfGoalState(self):
        for i in range(0, len(self.state), 4):
            side = self.state[i:i+4]
            if side.count(side[0]) != 4:
                return False
        return True

    # Function: Execute a single given Move to the State
    def executeMove(self, move):
        # For Invalid Move
        if move not in MOVES_LIST.keys():
            raise ValueError(f"Invalid Move: {move}")

        # Sticker Rotation for a Given Move
        permutations = MOVES_LIST[move]
        self.state = "".join([self.state[i] for i in permutations])
        return self.state

    # Function: Execute a set of given Moves to the State
    def executeMoves(self, moves):
        for move in moves.split():
            print(move)
            self.executeMove(move)

    # Function: Shuffle Cube with N Moves
    def shuffleCube(self, N):
        # Choose n random moves from the list of moves and creates an algorithm to apply
        moves = list(MOVES_LIST.keys())
        shuffle = " ".join([choice(moves) for i in range(N)])
        self.executeMoves(shuffle)


In [ ]:
def init(num_shuffle, depth_limit):
    observed_cubes = {} # map from cube state to depth
    actions_taken = []
    initial_cube = Cube2x2()
    initial_cube.shuffleCube(num_shuffle)
    return initial_cube, depth_limit, observed_cubes, actions_taken

def solve_cube(current_cube, ground_actions, current_depth, depth_limit, observed_cubes, actions_taken):
    show_status(current_cube, actions_taken, current_depth)

    # Check if depth limit is reached
    if current_depth >= depth_limit:
        return False

    # Check if current_cube is already observed, if observed, i.e., current_cube is the same as one of the observed_cubes,
    # if number of steps to reach current_cube is greater than or equal to the number of steps to reach the observed cube,
    # then we don't need to explore the current path, otherwise,
    # update observed_cubes with the minimum number of steps to
    # reach current_cube by setting it to current_depth and explore
    # the current path
    if (current_cube.returnCurrentState() in observed_cubes) and (observed_cubes[current_cube.returnCurrentState()] <= current_depth):
        return False
    else:
        observed_cubes[current_cube.returnCurrentState()] = current_depth

    # Check if current_cube is goal_cube
    if current_cube.checkIfGoalState():
        return actions_taken

    # Otherwise, choose an action and apply to the current_cube
    for action in ground_actions.keys():
        new_cube = Cube2x2(cube=current_cube.returnCurrentState())
        new_cube.executeMove(action)
        actions_taken_to_reach_current_cube = copy.deepcopy(actions_taken)
        actions_taken_to_reach_current_cube.append(action)
        result = solve_cube(new_cube, ground_actions, current_depth+1, depth_limit, observed_cubes, actions_taken_to_reach_current_cube)
        if result:
            return result
        else:
            continue

def show_status(current_cube, actions_taken, current_depth):
    print("========================================")
    print(f'Current Depth: {current_depth}')
    print(f'Current Cube State: {current_cube.returnCurrentState()}')
    print(f'Actions Taken: {actions_taken}')
    print()
    print()

def analyze(epochs, num_shuffles, depth_limits):
    assert len(num_shuffles) == len(depth_limits)
    average_running_time = []
    for i, num_shuffle in enumerate(num_shuffles):
        depth_limit = depth_limits[i]
        time_taken = []
        for _ in range(epochs):
            initial_cube, depth_limit, observed_cubes, actions_taken = init(num_shuffle, depth_limit)
            time_start = time.time()
            policy = solve_cube(initial_cube, MOVES_LIST, 0, depth_limit, observed_cubes, actions_taken)
            time_end = time.time()
            time_taken.append(time_end - time_start)
            print(f'Given {num_shuffle} shuffles and depth limit {depth_limit}, the policy is {policy}')
        time_taken_average = sum(time_taken)/epochs
        average_running_time.append(time_taken_average)
    visualize_performance(num_shuffles, depth_limits, average_running_time)

def visualize_performance(num_shuffles, depth_limits, average_running_time):
    plt.figure(figsize=(14, 7))
    plt.plot(num_shuffles, average_running_time, label="Average Running Time")
    plt.scatter(num_shuffles, average_running_time, alpha=0.5, edgecolors="w", s=30)
    plt.xticks(num_shuffles)
    plt.xlabel("Shuffles")
    plt.ylabel("Average Running Time")
    plt.title("DFS (Non-Heuristics) Performance")
    plt.show()

# Higher number of shuffles > 4 and depth limit will potentially overload your machine.
config = {
    'epochs': 10,
    'num_shuffles': [1, 2, 3, 4],
    'depth_limits': [2, 3, 4, 5]
}

analyze(config['epochs'], config['num_shuffles'], config['depth_limits'])

